# Part 3 - Create a New Parcel Record


The parcel fabric is a record-driven system. The parcel fabric organizes parcel data based on the form in which it was originally recorded. Parcel data is recorded on legal records such as plans, plats, deeds, and records of survey. Parcels are created and edited in response to changes in the legal record, and parcels are edited using record-driven workflows.

The parcel fabric preserves historic and parent parcels. By capturing the legal record that created or retired a parcel, the parcel lineage can be tracked in both directions. In an enterprise deployment, all edits are tracked, and the parcel fabric can be compared with its state in a previous moment in time.

Creating a parcel record requires the VersionManagementServer for branch versioning. However, it does not require access to the ParcelManager.  Creating a new record makes use of the ApplyEdits method of the REST API which is accessed through the FeatureServer endpoint.

In the ArcGIS Python API, this operation is simplified by using the `FeatureLayer.edit_features()` method.

This notebook will demonstrate:

1. Accessing the Records feature layer
2. Creating a branch version
3. Creating a parcel record with attributes using `FeatureLayer.edit_features()`

<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Part-3---Create-a-New-Parcel-Record" data-toc-modified-id="Part-3---Create-a-New-Parcel-Record-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Part 3 - Create a New Parcel Record</a></span><ul class="toc-item"><li><ul class="toc-item"><li><span><a href="#Branch-versioning-setup" data-toc-modified-id="Branch-versioning-setup-1.0.1"><span class="toc-item-num">1.0.1&nbsp;&nbsp;</span>Branch versioning setup</a></span></li><li><span><a href="#Create-a-new-feature-in-the-Records-feature-layer" data-toc-modified-id="Create-a-new-feature-in-the-Records-feature-layer-1.0.2"><span class="toc-item-num">1.0.2&nbsp;&nbsp;</span>Create a new feature in the Records feature layer</a></span><ul class="toc-item"><li><span><a href="#Prepare-the-attributes" data-toc-modified-id="Prepare-the-attributes-1.0.2.1"><span class="toc-item-num">1.0.2.1&nbsp;&nbsp;</span>Prepare the attributes</a></span></li><li><span><a href="#Access-the-Records-FeatureLayer" data-toc-modified-id="Access-the-Records-FeatureLayer-1.0.2.2"><span class="toc-item-num">1.0.2.2&nbsp;&nbsp;</span>Access the Records FeatureLayer</a></span></li><li><span><a href="#Insert-the-new-record-feature" data-toc-modified-id="Insert-the-new-record-feature-1.0.2.3"><span class="toc-item-num">1.0.2.3&nbsp;&nbsp;</span>Insert the new record feature</a></span></li></ul></li><li><span><a href="#API-Ref-Documentation" data-toc-modified-id="API-Ref-Documentation-1.0.3"><span class="toc-item-num">1.0.3&nbsp;&nbsp;</span>API Ref Documentation</a></span></li></ul></li></ul></li></ul></div>

In [1]:
from arcgis.gis import GIS
from arcgis.features.layer import FeatureLayerCollection
from arcgis.features.layer import FeatureLayer

In [2]:
base_server_url = "https://myserver.domain.com/web_adaptor/rest/services/WashingtonCountyLSA/"
gis = GIS("https://myserver.domain.com/web_adaptor/", "username", "pass.word", verify_cert=False)

# Generate the enpoint urls for feature server, version management and parcel fabric
service_endpoints = ["FeatureServer", "VersionManagementServer"]                    
service_urls = {url: base_server_url + url for url in service_endpoints}

### Branch versioning setup
- Get the Parcel Fabric FeatureLayerCollection (FLC)
- Get the VersionManagementServer object from the FLC
- Create a new branch version
- Get the fully qualified (owner.name) version name

In [3]:
parcel_fabric_flc = FeatureLayerCollection(service_urls["FeatureServer"], gis)
version_management_server = parcel_fabric_flc.versions

new_version_name = "new_record_version"
version_management_server.get(f"admin.{new_version_name}").delete()

version_management_server.create(new_version_name)
fq_version_name = version_management_server.get(f"admin.{new_version_name}").properties.versionName
fq_version_name

'admin.new_record_version'

### Create a new feature in the Records feature layer
- Get access to the correct feature layer
- Create a dict containing some attributes for the new record
- Get the FeatureLayer
- Use `FeatureLayer.edit_features` to create the new feature and insert the attributes.

<div class="alert alert-info">
    <b>Note:</b> It is expected that the new parcel record will contain an empty geometry.  When parcel features are attributed with this parcel record GUID, the Parcel Fabric Build function will generate a shape consisting of the outer boundaries of the associated parcel features.
</div>

In [1]:
# Get the URL of the Records feature layer
records_fl_url = [n for n in parcel_fabric_flc.layers if n.properties.name == "Records"]
records_fl_url = records_fl_url[0].url
records_fl_url

https://myserver.domain.com/web_adaptor/rest/services/WashingtonCountyLSA/FeatureServer/1


#### Prepare the attributes

In [5]:
# A name for the new record
new_record_name = "ParcelRecord001"

# Record information with empty geometry.  The geometry is created during Build
record_dict = {
    "attributes": 
        {"name": new_record_name},
    "geometry": None
    }

record_dict

{'attributes': {'name': 'ParcelRecord001'}, 'geometry': None}

#### Access the Records FeatureLayer

In [2]:
records_fl = FeatureLayer(records_fl_url, gis)
records_fl

<FeatureLayer url:"https://myserver.domain.com/web_adaptor/rest/services/WashingtonCountyLSA/FeatureServer/1">


#### Insert the new record feature

In [7]:
# Create the new record within the new branch version
new_record = records_fl.edit_features(
                            adds=[record_dict], 
                            gdb_version=fq_version_name)

new_record

{'addResults': [{'objectId': 52130,
   'globalId': '{A82773D1-ACEB-4391-AF35-7D4873B8B1BA}',
   'success': True}],
 'updateResults': [],
 'deleteResults': []}

### API Ref Documentation

- [ArcGIS Python API - Version Manager](https://developers.arcgis.com/python/api-reference/arcgis.features.managers.html#versionmanager)
- [ArcGIS Python API - Parcel Fabric Manager](https://developers.arcgis.com/python/api-reference/arcgis.features.managers.html#parcelfabricmanager)
- [ArcGIS REST API - VersionManagementServer](https://developers.arcgis.com/rest/services-reference/enterprise/version-management-service.htm)
- [ArcGIS REST API - ParcelFabricServer](https://developers.arcgis.com/rest/services-reference/enterprise/overview-of-parcel-fabric-sevices.htm)
- [ArcGIS Pro - Branch Versioning Scenarios](https://pro.arcgis.com/en/pro-app/latest/help/data/geodatabases/overview/branch-version-scenarios.htm)